# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fd4354fa2b0>
├── 'a' --> tensor([[ 0.3823,  0.9901,  0.9891],
│                   [ 0.7910, -2.4117, -1.0424]])
└── 'x' --> <FastTreeValue 0x7fd43046b400>
    └── 'c' --> tensor([[-0.6880,  0.3536,  0.3410,  0.3584],
                        [ 1.4188, -0.7418, -1.6623, -1.5132],
                        [-1.7965,  1.5446, -0.9980, -0.1302]])

In [4]:
t.a

tensor([[ 0.3823,  0.9901,  0.9891],
        [ 0.7910, -2.4117, -1.0424]])

In [5]:
%timeit t.a

67.5 ns ± 0.0817 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fd4354fa2b0>
├── 'a' --> tensor([[ 0.4659,  0.2434,  1.3725],
│                   [ 0.7374, -1.4128, -1.3578]])
└── 'x' --> <FastTreeValue 0x7fd43046b400>
    └── 'c' --> tensor([[-0.6880,  0.3536,  0.3410,  0.3584],
                        [ 1.4188, -0.7418, -1.6623, -1.5132],
                        [-1.7965,  1.5446, -0.9980, -0.1302]])

In [7]:
%timeit t.a = new_value

72.8 ns ± 0.0332 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.3823,  0.9901,  0.9891],
               [ 0.7910, -2.4117, -1.0424]]),
    x: Batch(
           c: tensor([[-0.6880,  0.3536,  0.3410,  0.3584],
                      [ 1.4188, -0.7418, -1.6623, -1.5132],
                      [-1.7965,  1.5446, -0.9980, -0.1302]]),
       ),
)

In [10]:
b.a

tensor([[ 0.3823,  0.9901,  0.9891],
        [ 0.7910, -2.4117, -1.0424]])

In [11]:
%timeit b.a

58.9 ns ± 0.0791 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.4942, -0.4816, -0.7181],
               [ 0.8403, -1.4860, -0.7209]]),
    x: Batch(
           c: tensor([[-0.6880,  0.3536,  0.3410,  0.3584],
                      [ 1.4188, -0.7418, -1.6623, -1.5132],
                      [-1.7965,  1.5446, -0.9980, -0.1302]]),
       ),
)

In [13]:
%timeit b.a = new_value

497 ns ± 0.177 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

817 ns ± 0.122 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.5 µs ± 35.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

142 µs ± 262 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

142 µs ± 745 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fd365783520>
├── 'a' --> tensor([[[ 0.3823,  0.9901,  0.9891],
│                    [ 0.7910, -2.4117, -1.0424]],
│           
│                   [[ 0.3823,  0.9901,  0.9891],
│                    [ 0.7910, -2.4117, -1.0424]],
│           
│                   [[ 0.3823,  0.9901,  0.9891],
│                    [ 0.7910, -2.4117, -1.0424]],
│           
│                   [[ 0.3823,  0.9901,  0.9891],
│                    [ 0.7910, -2.4117, -1.0424]],
│           
│                   [[ 0.3823,  0.9901,  0.9891],
│                    [ 0.7910, -2.4117, -1.0424]],
│           
│                   [[ 0.3823,  0.9901,  0.9891],
│                    [ 0.7910, -2.4117, -1.0424]],
│           
│                   [[ 0.3823,  0.9901,  0.9891],
│                    [ 0.7910, -2.4117, -1.0424]],
│           
│                   [[ 0.3823,  0.9901,  0.9891],
│                    [ 0.7910, -2.4117, -1.0424]]])
└── 'x' --> <FastTreeValue 0x7fd365783910>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.6 µs ± 28.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fd3657efb80>
├── 'a' --> tensor([[ 0.3823,  0.9901,  0.9891],
│                   [ 0.7910, -2.4117, -1.0424],
│                   [ 0.3823,  0.9901,  0.9891],
│                   [ 0.7910, -2.4117, -1.0424],
│                   [ 0.3823,  0.9901,  0.9891],
│                   [ 0.7910, -2.4117, -1.0424],
│                   [ 0.3823,  0.9901,  0.9891],
│                   [ 0.7910, -2.4117, -1.0424],
│                   [ 0.3823,  0.9901,  0.9891],
│                   [ 0.7910, -2.4117, -1.0424],
│                   [ 0.3823,  0.9901,  0.9891],
│                   [ 0.7910, -2.4117, -1.0424],
│                   [ 0.3823,  0.9901,  0.9891],
│                   [ 0.7910, -2.4117, -1.0424],
│                   [ 0.3823,  0.9901,  0.9891],
│                   [ 0.7910, -2.4117, -1.0424]])
└── 'x' --> <FastTreeValue 0x7fd43043b1f0>
    └── 'c' --> tensor([[-0.6880,  0.3536,  0.3410,  0.3584],
                        [ 1.4188, -0.7418, -1.6623, -1.5132],
                 

In [23]:
%timeit t_cat(trees)

30.5 µs ± 47.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

67.6 µs ± 73.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.3823,  0.9901,  0.9891],
                [ 0.7910, -2.4117, -1.0424]],
       
               [[ 0.3823,  0.9901,  0.9891],
                [ 0.7910, -2.4117, -1.0424]],
       
               [[ 0.3823,  0.9901,  0.9891],
                [ 0.7910, -2.4117, -1.0424]],
       
               [[ 0.3823,  0.9901,  0.9891],
                [ 0.7910, -2.4117, -1.0424]],
       
               [[ 0.3823,  0.9901,  0.9891],
                [ 0.7910, -2.4117, -1.0424]],
       
               [[ 0.3823,  0.9901,  0.9891],
                [ 0.7910, -2.4117, -1.0424]],
       
               [[ 0.3823,  0.9901,  0.9891],
                [ 0.7910, -2.4117, -1.0424]],
       
               [[ 0.3823,  0.9901,  0.9891],
                [ 0.7910, -2.4117, -1.0424]]]),
    x: Batch(
           c: tensor([[[-0.6880,  0.3536,  0.3410,  0.3584],
                       [ 1.4188, -0.7418, -1.6623, -1.5132],
                       [-1.7965,  1.5446, -0.9980, -0.1302]],
         

In [26]:
%timeit Batch.stack(batches)

79.7 µs ± 356 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.3823,  0.9901,  0.9891],
               [ 0.7910, -2.4117, -1.0424],
               [ 0.3823,  0.9901,  0.9891],
               [ 0.7910, -2.4117, -1.0424],
               [ 0.3823,  0.9901,  0.9891],
               [ 0.7910, -2.4117, -1.0424],
               [ 0.3823,  0.9901,  0.9891],
               [ 0.7910, -2.4117, -1.0424],
               [ 0.3823,  0.9901,  0.9891],
               [ 0.7910, -2.4117, -1.0424],
               [ 0.3823,  0.9901,  0.9891],
               [ 0.7910, -2.4117, -1.0424],
               [ 0.3823,  0.9901,  0.9891],
               [ 0.7910, -2.4117, -1.0424],
               [ 0.3823,  0.9901,  0.9891],
               [ 0.7910, -2.4117, -1.0424]]),
    x: Batch(
           c: tensor([[-0.6880,  0.3536,  0.3410,  0.3584],
                      [ 1.4188, -0.7418, -1.6623, -1.5132],
                      [-1.7965,  1.5446, -0.9980, -0.1302],
                      [-0.6880,  0.3536,  0.3410,  0.3584],
                      [ 1.4188, -

In [28]:
%timeit Batch.cat(batches)

150 µs ± 281 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

348 µs ± 1.84 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
